#Importo Librerie

In [1]:
import pandas as pd                                          #importo libreria pandas per gestire dati in formato DataFrame
import numpy as np                                           # importo libreria numpy per operare su array multidimensionali
import random                                                #permette di generare numeri casuali

#Carico csv e creazione dataFrame

Creo due dataFrame movies e ratings

In [2]:
#Creo due dataFrame movies e ratings

movies = pd.read_csv(r'/content/movies.csv', sep=',', names=['movieId', 'title', 'genres'],  encoding="Latin1")

# pd.read_csv: Utilizza la funzione read_csv della libreria pandas per leggere i dati da un file CSV.
# "movies.csv": Specifica il percorso del file CSV.
# sep=',': Specifica che il separatore tra le colonne nel file CSV è la virgola.
# names=['movieId', 'title', 'genres']: Assegna i nomi alle colonne del DataFrame.
# encoding="Latin1": Specifica l'encoding del file. Latin1 è comune per gestire caratteri speciali in testi in lingue europee.

ratings = pd.read_csv(r'/content/ratings.csv', sep=',', names =['userId', 'movieId', 'rating', 'timestamp'], encoding = "Latin1")


Visualizzo movies

In [3]:
movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


visualizzo ratings

In [4]:
ratings.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


# Elimino le colonne che non interessano (genres),(timestamp)

In [5]:
movies.drop(['genres'], axis=1, inplace=True)
#['genres']: Specifica il nome della colonna da rimuovere, che è 'genres' in questo caso.
#axis=1: Indica che la rimozione deve avvenire lungo l'asse delle colonne.
#inplace=True: Indica che la modifica deve essere effettuata direttamente sul DataFrame
ratings.drop(['timestamp'], axis=1, inplace=True)

In [6]:
movies.head()

,movieId,title
0,1,Toy Story (1995)
1,2,Jumanji (1995)
2,3,Grumpier Old Men (1995)
3,4,Waiting to Exhale (1995)
4,5,Father of the Bride Part II (1995)


In [7]:
ratings.head()

,userId,movieId,rating
0,1,1,4.0
1,1,3,4.0
2,1,6,4.0
3,1,47,5.0
4,1,50,5.0


Effettuo la media dei voti per ogni singolo user

In [8]:
mean = ratings.groupby(by="userId",as_index=False)['rating'].mean()
mean.head()


,userId,rating
0,1,4.366379
1,2,3.948276
2,3,2.435897
3,4,3.555556
4,5,3.636364


#Normalizzazione df e creazione tabella pivot

In [9]:
#Normalizzazione dei voti dei singoli utenti
#faccio la fusione dei due dataframe di ratings e mean sulla base dell'userId
rating_avg = pd.merge(ratings,mean,on='userId')
rating_avg.head()

,userId,movieId,rating_x,rating_y
0,1,1,4.0,4.366379
1,1,3,4.0,4.366379
2,1,6,4.0,4.366379
3,1,47,5.0,4.366379
4,1,50,5.0,4.366379


Creo una nuova colonna adg_rating che deve essere la differenza tra il voto dato ad un movie e la media dei voti dati dall'utente


In [10]:
rating_avg = rating_avg.rename(columns={"rating_y": "rating_avg"})
rating_avg['adg_rating']=rating_avg['rating_x']-rating_avg['rating_avg']
rating_avg.head()

,userId,movieId,rating_x,rating_avg,adg_rating
0,1,1,4.0,4.366379,-0.366379
1,1,3,4.0,4.366379,-0.366379
2,1,6,4.0,4.366379,-0.366379
3,1,47,5.0,4.366379,0.633621
4,1,50,5.0,4.366379,0.633621


Creo una tabella pivot con indice l'user id e righe movieId, andando ad inserire come valori i voti e ottenendo NaN per i film che non sono stati votati dagli n utenti

In [11]:
check = pd.pivot_table(rating_avg,values='rating_x',index='userId',columns='movieId')
check.head()

movieId,1,2,3,4,5,6,7,8,9,10,...,193565,193567,193571,193573,193579,193581,193583,193585,193587,193609
userId,,,,,,,,,,,,,,,,,,,,,
1,4.0,NaN,4.0,NaN,NaN,4.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Opero sulla tabella Pivot

In [12]:
# Calcola la media dei voti per ciascun film
movie_avg_ratings = ratings.groupby('movieId')['rating'].mean()

# Riempi i valori NaN nella tabella pivot con le medie dei voti dei rispettivi film
check_filled = check.apply(lambda col: col.fillna(movie_avg_ratings[col.name]), axis=0)

# Ora 'check_filled' contiene la tabella pivot con i valori NaN sostituiti dalle medie dei voti per ciascun film
check_filled

movieId,1,2,3,4,5,6,7,8,9,10,...,193565,193567,193571,193573,193579,193581,193583,193585,193587,193609
userId,,,,,,,,,,,,,,,,,,,,,
1,4.00000,3.431818,4.000000,2.357143,3.071429,4.000000,3.185185,2.875,3.125,3.496212,...,3.5,3.0,4.0,4.0,3.5,4.0,3.5,3.5,3.5,4.0
2,3.92093,3.431818,3.259615,2.357143,3.071429,3.946078,3.185185,2.875,3.125,3.496212,...,3.5,3.0,4.0,4.0,3.5,4.0,3.5,3.5,3.5,4.0
3,3.92093,3.431818,3.259615,2.357143,3.071429,3.946078,3.185185,2.875,3.125,3.496212,...,3.5,3.0,4.0,4.0,3.5,4.0,3.5,3.5,3.5,4.0
4,3.92093,3.431818,3.259615,2.357143,3.071429,3.946078,3.185185,2.875,3.125,3.496212,...,3.5,3.0,4.0,4.0,3.5,4.0,3.5,3.5,3.5,4.0
5,4.00000,3.431818,3.259615,2.357143,3.071429,3.946078,3.185185,2.875,3.125,3.496212,...,3.5,3.0,4.0,4.0,3.5,4.0,3.5,3.5,3.5,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
606,2.50000,3.431818,3.259615,2.357143,3.071429,3.946078,2.500000,2.875,3.125,3.496212,...,3.5,3.0,4.0,4.0,3.5,4.0,3.5,3.5,3.5,4.0
607,4.00000,3.431818,3.259615,2.357143,3.071429,3.946078,3.185185,2.875,3.125,3.496212,...,3.5,3.0,4.0,4.0,3.5,4.0,3.5,3.5,3.5,4.0
608,2.50000,2.000000,2.000000,2.357143,3.071429,3.946078,3.185185,2.875,3.125,4.000000,...,3.5,3.0,4.0,4.0,3.5,4.0,3.5,3.5,3.5,4.0


#Randomizzazione degli utenti su cui applicare le strategie

Estraggo casualmente 5 righe

In [13]:
# Estrai casualmente 5 righe dalla tabella check_filled
random_rows = check_filled.sample(n=5)

# Stampa le 5 righe estratte casualmente
random_rows


movieId,1,2,3,4,5,6,7,8,9,10,...,193565,193567,193571,193573,193579,193581,193583,193585,193587,193609
userId,,,,,,,,,,,,,,,,,,,,,
479,3.92093,3.431818,3.259615,2.357143,3.071429,3.946078,3.185185,2.875,3.125,3.496212,...,3.5,3.0,4.0,4.0,3.5,4.0,3.5,3.5,3.5,4.0
276,4.00000,4.000000,3.259615,2.357143,3.000000,3.946078,4.000000,2.875,3.125,3.496212,...,3.5,3.0,4.0,4.0,3.5,4.0,3.5,3.5,3.5,4.0
129,3.92093,3.431818,3.259615,2.357143,3.071429,3.946078,3.185185,2.875,3.125,3.496212,...,3.5,3.0,4.0,4.0,3.5,4.0,3.5,3.5,3.5,4.0
426,3.92093,4.500000,3.259615,2.357143,3.071429,3.946078,3.185185,2.875,3.125,3.496212,...,3.5,3.0,4.0,4.0,3.5,4.0,3.5,3.5,3.5,4.0
69,3.92093,3.431818,3.259615,2.357143,3.071429,3.946078,3.185185,2.875,3.125,3.496212,...,3.5,3.0,4.0,4.0,3.5,4.0,3.5,3.5,3.5,4.0
606,2.50000,3.431818,3.259615,2.357143,3.071429,3.946078,2.500000,2.875,3.125,3.496212,...,3.5,3.0,4.0,4.0,3.5,4.0,3.5,3.5,3.5,4.0
248,3.92093,3.431818,3.259615,2.357143,3.071429,3.946078,3.185185,2.875,3.125,3.500000,...,3.5,3.0,4.0,4.0,3.5,4.0,3.5,3.5,3.5,4.0


Normalizzo la tabella secondo la media di voto di ogni utente

In [14]:
def normalize_table_by_user_mean(table):
    # Calcola la media dei voti per ogni utente
    user_means = table.mean(axis=1)

    # Sottrai la media dell'utente da ogni valore nella tabella
    normalized_table = table.sub(user_means, axis=0)

    return normalized_table

# Esempio di utilizzo con la tabella random_rows
normalized_table = normalize_table_by_user_mean(random_rows)

# Stampa la tabella normalizzata
normalized_table

movieId,1,2,3,4,5,6,7,8,9,10,...,193565,193567,193571,193573,193579,193581,193583,193585,193587,193609
userId,,,,,,,,,,,,,,,,,,,,,
479,0.658514,0.169402,-0.002801,-0.905273,-0.190988,0.683662,-0.077231,-0.387416,-0.137416,0.233796,...,0.237584,-0.262416,0.737584,0.737584,0.237584,0.737584,0.237584,0.237584,0.237584,0.737584
276,0.733201,0.733201,-0.007183,-0.909656,-0.266799,0.679280,0.733201,-0.391799,-0.141799,0.229414,...,0.233201,-0.266799,0.733201,0.733201,0.233201,0.733201,0.233201,0.233201,0.233201,0.733201
129,0.653691,0.164579,-0.007624,-0.910096,-0.195810,0.678840,-0.082054,-0.392239,-0.142239,0.228973,...,0.232761,-0.267239,0.732761,0.732761,0.232761,0.732761,0.232761,0.232761,0.232761,0.732761
426,0.658248,1.237317,-0.003067,-0.905540,-0.191254,0.683396,-0.077497,-0.387683,-0.137683,0.233530,...,0.237317,-0.262683,0.737317,0.737317,0.237317,0.737317,0.237317,0.237317,0.237317,0.737317
69,0.655263,0.166151,-0.006052,-0.908525,-0.194239,0.680411,-0.080482,-0.390668,-0.140668,0.230545,...,0.234332,-0.265668,0.734332,0.734332,0.234332,0.734332,0.234332,0.234332,0.234332,0.734332
606,-0.770670,0.161148,-0.011055,-0.913527,-0.199241,0.675408,-0.770670,-0.395670,-0.145670,0.225542,...,0.229330,-0.270670,0.729330,0.729330,0.229330,0.729330,0.229330,0.229330,0.229330,0.729330
248,0.658902,0.169790,-0.002413,-0.904885,-0.190600,0.684050,-0.076843,-0.387028,-0.137028,0.237972,...,0.237972,-0.262028,0.737972,0.737972,0.237972,0.737972,0.237972,0.237972,0.237972,0.737972


#Utilizzo strategia Average

La strategia prevede per ogni item di calcolare la media dei voti assegnata da tutti gli user. La media viene calcolata in una nuova, Ho applicato la stretegia sia al df normalizzato che non normalizzato, confrontando i risultati.

## Su df non Normalizzato

In [15]:
def order_items_by_average(ratings):
    # Copia il DataFrame originale per non alterare i dati originali
    ratings_copy = ratings.copy()

    # Calcola la media per ogni item (sull'asse delle righe)
    item_means = ratings_copy.mean(axis=1)

    # Arrotonda i valori delle medie a una cifra decimale
    item_means_rounded = item_means.round(1)

    # Aggiunge una nuova colonna "average" al DataFrame con le medie arrotondate
    ratings_copy['average'] = item_means_rounded

    # Stampa per visualizzare la tabella
    print(ratings_copy)

    # Ordinare gli item in base agli average in ordine decrescente
    sorted_items = item_means.sort_values(ascending=False)

    # Crea la lista risultato
    result_list = []

    # Aggiunge gli ID degli item alla lista in ordine decrescente
    current_value = None

    for item in sorted_items.index:
        value = item_means[item]
        if current_value is None or value == current_value:
            result_list.append(item)
        current_value = value

    return result_list[:10]

# Esempio di utilizzo
result = order_items_by_average(random_rows.T)
print(result)


userId        479       276       129       426        69       606       248  \
movieId                                                                         
1        3.920930  4.000000  3.920930  3.920930  3.920930  2.500000  3.920930   
2        3.431818  4.000000  3.431818  4.500000  3.431818  3.431818  3.431818   
3        3.259615  3.259615  3.259615  3.259615  3.259615  3.259615  3.259615   
4        2.357143  2.357143  2.357143  2.357143  2.357143  2.357143  2.357143   
5        3.071429  3.000000  3.071429  3.071429  3.071429  3.071429  3.071429   
...           ...       ...       ...       ...       ...       ...       ...   
193581   4.000000  4.000000  4.000000  4.000000  4.000000  4.000000  4.000000   
193583   3.500000  3.500000  3.500000  3.500000  3.500000  3.500000  3.500000   
193585   3.500000  3.500000  3.500000  3.500000  3.500000  3.500000  3.500000   
193587   3.500000  3.500000  3.500000  3.500000  3.500000  3.500000  3.500000   
193609   4.000000  4.000000 

In [16]:
result_id = result
result_id

[2196, 64501, 138966, 163925, 138835, 64499, 83969, 138632, 27523, 164367]

In [17]:
def get_titles_from_ids(result_id, movies):
    titles = [movies.loc[movies['movieId'] == id_, 'title'].values[0] for id_ in result_id]
    return titles

# Esempio di utilizzo
titles = get_titles_from_ids(result_id, movies)
print(titles)


['Knock Off (1998)', 'Che: Part Two (2008)', 'Nasu: Summer in Andalusia (2003)', 'Wings, Legs and Tails (1986)', 'Return to Treasure Island (1988)', 'Che: Part One (2008)', 'Down Argentine Way (1940)', 'Tokyo Tribe (2014)', 'My Sassy Girl (Yeopgijeogin geunyeo) (2001)', 'The Girl with All the Gifts (2016)']


## Su df normalizzato

In [18]:
def order_items_by_average_normalizated(ratings_inverted):
    # Copia il DataFrame originale per non alterare i dati originali
    ratings_inverted_copy = ratings_inverted.copy()

    # Calcola la media per ogni item (sull'asse delle righe)
    item_means = ratings_inverted_copy.mean(axis=1)

    # Arrotonda i valori delle medie a una cifra decimale
    item_means_rounded = item_means.round(1)

    # Aggiunge una nuova colonna "average" al DataFrame con le medie arrotondate
    ratings_inverted_copy['average'] = item_means_rounded

    # Stampa per visualizzare la tabella
    print(ratings_inverted_copy)

    # Ordinare gli item in base agli average in ordine decrescente
    sorted_items = item_means.sort_values(ascending=False)

    # Crea la lista risultato
    result_list = []

    # Aggiunge gli ID degli item alla lista in ordine decrescente
    current_value = None

    for item in sorted_items.index:
        value = item_means[item]
        if current_value is None or value == current_value:
            result_list.append(item)
        current_value = value

    return result_list[:10]

# Esempio di utilizzo
result = order_items_by_average_normalizated(normalized_table.T)
print(result)


userId        479       276       129       426        69       606       248  \
movieId                                                                         
1        0.658514  0.733201  0.653691  0.658248  0.655263 -0.770670  0.658902   
2        0.169402  0.733201  0.164579  1.237317  0.166151  0.161148  0.169790   
3       -0.002801 -0.007183 -0.007624 -0.003067 -0.006052 -0.011055 -0.002413   
4       -0.905273 -0.909656 -0.910096 -0.905540 -0.908525 -0.913527 -0.904885   
5       -0.190988 -0.266799 -0.195810 -0.191254 -0.194239 -0.199241 -0.190600   
...           ...       ...       ...       ...       ...       ...       ...   
193581   0.737584  0.733201  0.732761  0.737317  0.734332  0.729330  0.737972   
193583   0.237584  0.233201  0.232761  0.237317  0.234332  0.229330  0.237972   
193585   0.237584  0.233201  0.232761  0.237317  0.234332  0.229330  0.237972   
193587   0.237584  0.233201  0.232761  0.237317  0.234332  0.229330  0.237972   
193609   0.737584  0.733201 

In [19]:
result_id_normalized = result
result_id_normalized

[2196, 64501, 138966, 163925, 138835, 64499, 83969, 138632, 27523, 164367]

In [20]:
def get_titles_from_ids(result_id, movies):
    titles = [movies.loc[movies['movieId'] == id_, 'title'].values[0] for id_ in result_id]
    return titles

# Esempio di utilizzo
titles = get_titles_from_ids(result_id_normalized, movies)
print(titles)


['Knock Off (1998)', 'Che: Part Two (2008)', 'Nasu: Summer in Andalusia (2003)', 'Wings, Legs and Tails (1986)', 'Return to Treasure Island (1988)', 'Che: Part One (2008)', 'Down Argentine Way (1940)', 'Tokyo Tribe (2014)', 'My Sassy Girl (Yeopgijeogin geunyeo) (2001)', 'The Girl with All the Gifts (2016)']


In [21]:
# Confronto se le due liste sono uguali
if result_id == result_id_normalized:
    print("Le liste sono uguali.")
else:
    print("Le liste non sono uguali.")

Le liste sono uguali.


#Utilizzo Strategia Average Without Misery

Questa strategia si basa sul calcolo della media dei rating di ogni singolo item, ma senza considerare gli item che hanno uno score sotto una determinata soglia. Gli item il cui voto non supera la soglia nella matrice hanno valore NaN. Ho applicato questa strategia sia al df normalizzato che normale

## Su df non Normalizzato





In [22]:
import pandas as pd

def order_items_by_average_with_threshold(ratings, threshold):
    # Copia il DataFrame originale per non alterare i dati originali
    ratings_copy = ratings.copy()

    # Applica la soglia alle valutazioni
    ratings_copy[ratings_copy < threshold] = pd.NA

    # Trova item con almeno una valutazione al di sotto della soglia
    items_below_threshold = ratings_copy[ratings_copy.isna().any(axis=1)].index.tolist()

    # Calcola la media per ogni item (sull'asse delle righe)
    item_means = ratings_copy.mean(axis=1)

    # Arrotonda i valori delle medie a una cifra decimale
    item_means_rounded = item_means.round(1)

    # Aggiunge una nuova colonna "average" al DataFrame con le medie arrotondate
    ratings_copy['average'] = item_means_rounded

    # Stampa per visualizzare la tabella
    print(ratings_copy)

    # Escludi gli item con almeno una valutazione al di sotto della soglia
    sorted_items = item_means[~item_means.index.isin(items_below_threshold)].sort_values(ascending=False)

    # Crea la lista risultato
    result_list = []

    # Aggiunge gli ID degli item alla lista in ordine decrescente con parentesi tonde per gli uguali
    current_value = None

    for item in sorted_items.index:
        value = item_means[item]
        if current_value is None or value == current_value:
            result_list.append(item)
        current_value = value

    return result_list[:10]

# Esempio di utilizzo

threshold = 4
result = order_items_by_average_with_threshold(random_rows.T, threshold)
print(result)


userId   479  276  129  426   69  606  248  average
movieId                                            
1        NaN  4.0  NaN  NaN  NaN  NaN  NaN      4.0
2        NaN  4.0  NaN  4.5  NaN  NaN  NaN      4.2
3        NaN  NaN  NaN  NaN  NaN  NaN  NaN      NaN
4        NaN  NaN  NaN  NaN  NaN  NaN  NaN      NaN
5        NaN  NaN  NaN  NaN  NaN  NaN  NaN      NaN
...      ...  ...  ...  ...  ...  ...  ...      ...
193581   4.0  4.0  4.0  4.0  4.0  4.0  4.0      4.0
193583   NaN  NaN  NaN  NaN  NaN  NaN  NaN      NaN
193585   NaN  NaN  NaN  NaN  NaN  NaN  NaN      NaN
193587   NaN  NaN  NaN  NaN  NaN  NaN  NaN      NaN
193609   4.0  4.0  4.0  4.0  4.0  4.0  4.0      4.0

[9724 rows x 8 columns]
[147250, 172875, 4116, 147410, 26147, 3951, 3942, 3941, 3940, 3939]


In [23]:
result_id = result
result_id

[147250, 172875, 4116, 147410, 26147, 3951, 3942, 3941, 3940, 3939]

In [24]:
def get_titles_from_ids(result_id, movies):
    titles = [movies.loc[movies['movieId'] == id_, 'title'].values[0] for id_ in result_id]
    return titles

# Esempio di utilizzo
titles = get_titles_from_ids(result, movies)
print(titles)


['The Adventures of Sherlock Holmes and Doctor Watson', 'A Detective Story (2003)', 'Hollywood Shuffle (1987)', 'A Perfect Day (2015)', 'Thousand Clowns, A (1965)', 'Two Family House (2000)', 'Sorority House Massacre II (1990)', 'Sorority House Massacre (1986)', 'Slumber Party Massacre III (1990)', 'Slumber Party Massacre II (1987)']


## Su df Normalizzato

In [25]:
import pandas as pd

def order_items_by_average_with_threshold(ratings, threshold):
    # Copia il DataFrame originale per non alterare i dati originali
    ratings_copy = ratings.copy()

    # Applica la soglia alle valutazioni
    ratings_copy[ratings_copy < threshold] = pd.NA

    # Trova item con almeno una valutazione al di sotto della soglia
    items_below_threshold = ratings_copy[ratings_copy.isna().any(axis=1)].index.tolist()

    # Calcola la media per ogni item (sull'asse delle righe)
    item_means = ratings_copy.mean(axis=1)

    # Arrotonda i valori delle medie a una cifra decimale
    item_means_rounded = item_means.round(1)

    # Aggiunge una nuova colonna "average" al DataFrame con le medie arrotondate
    ratings_copy['average'] = item_means_rounded

    # Stampa per visualizzare la tabella
    print(ratings_copy)

    # Escludi gli item con almeno una valutazione al di sotto della soglia
    sorted_items = item_means[~item_means.index.isin(items_below_threshold)].sort_values(ascending=False)

    # Crea la lista risultato
    result_list = []

    # Aggiunge gli ID degli item alla lista in ordine decrescente con parentesi tonde per gli uguali
    current_value = None

    for item in sorted_items.index:
        value = item_means[item]
        if current_value is None or value == current_value:
            result_list.append(item)
        current_value = value

    return result_list[:10]

# Esempio di utilizzo

threshold = 0.6
result = order_items_by_average_with_threshold(normalized_table.T, threshold)
print(result)


userId        479       276       129       426        69      606       248  \
movieId                                                                        
1        0.658514  0.733201  0.653691  0.658248  0.655263      NaN  0.658902   
2             NaN  0.733201       NaN  1.237317       NaN      NaN       NaN   
3             NaN       NaN       NaN       NaN       NaN      NaN       NaN   
4             NaN       NaN       NaN       NaN       NaN      NaN       NaN   
5             NaN       NaN       NaN       NaN       NaN      NaN       NaN   
...           ...       ...       ...       ...       ...      ...       ...   
193581   0.737584  0.733201  0.732761  0.737317  0.734332  0.72933  0.737972   
193583        NaN       NaN       NaN       NaN       NaN      NaN       NaN   
193585        NaN       NaN       NaN       NaN       NaN      NaN       NaN   
193587        NaN       NaN       NaN       NaN       NaN      NaN       NaN   
193609   0.737584  0.733201  0.732761  0

In [26]:
result_id_normalized = result
result_id_normalized

[53578, 44851, 1349, 131610, 1310, 107771, 131724, 147410, 2969, 4402]

In [27]:
def get_titles_from_ids(result_id, movies):
    titles = [movies.loc[movies['movieId'] == id_, 'title'].values[0] for id_ in result_id]
    return titles

# Esempio di utilizzo
titles = get_titles_from_ids(result_id_normalized, movies)
print(titles)


['Valet, The (La doublure) (2006)', 'Go for Zucker! (Alles auf Zucker!) (2004)', 'Vampire in Venice (Nosferatu a Venezia) (Nosferatu in Venice) (1986)', 'Willy/Milly (1986)', 'Hype! (1996)', 'Only Lovers Left Alive (2013)', 'The Jinx: The Life and Deaths of Robert Durst (2015)', 'A Perfect Day (2015)', 'Man and a Woman, A (Un homme et une femme) (1966)', 'Dr. Goldfoot and the Bikini Machine (1965)']


In [28]:
# Confronto se le due liste sono uguali
if result_id == result_id_normalized:
    print("Le liste sono uguali.")
else:
    print("Le liste non sono uguali.")

Le liste non sono uguali.


#Least Misery

Strategia che prevede la creazione di una riga di ratings con il minimo dei rating individuali per ogni item. Gli item sono selezionati in base all’ordine di rating di questa nuova riga, prima l’item con indice più alto e successivamente a descrescere.

##Su df non Normalizzato

In [29]:
import pandas as pd

def order_items_by_least_misery(random_rows):
    # Copia il DataFrame originale per non alterare i dati originali
    random_rows_copy = random_rows.copy()

    # Calcolare il voto minore per ogni ItemId tra le tre persone
    min_scores = random_rows_copy.min(axis=0)

    # Trovare gli ItemId con lo stesso voto minimo
    equal_min_scores = min_scores[min_scores.duplicated(keep=False)].index.tolist()

    # Aggiungere la riga "Minimo" al DataFrame con il rating minore corrispondente ad ogni ItemId
    random_rows_copy.loc['Minimo'] = min_scores

    # Stampa per visualizzare la tabella
    print(random_rows_copy)

    # Creare la lista risultato
    result_list = []

 # Aggiungere gli ItemId alla lista in ordine decrescente
    for item in min_scores.index:
            result_list.append(item)

    # Ordinare la lista risultato in base all'ordinamento di sorted_items
    sorted_items = min_scores.sort_values(ascending=False)
    result_list.sort(key=lambda x: sorted_items.index.get_loc(x))

    # Ordinare la lista risultato in base all'ordinamento di sorted_items
    sorted_items = min_scores.sort_values(ascending=False)
    result_list.sort(key=lambda x: sorted_items.index.get_loc(x))

    return result_list[:10]

# Esempio di utilizzo

result = order_items_by_least_misery(random_rows)
print(result)


movieId   1         2         3         4         5         6         7       \
userId                                                                         
479      3.92093  3.431818  3.259615  2.357143  3.071429  3.946078  3.185185   
276      4.00000  4.000000  3.259615  2.357143  3.000000  3.946078  4.000000   
129      3.92093  3.431818  3.259615  2.357143  3.071429  3.946078  3.185185   
426      3.92093  4.500000  3.259615  2.357143  3.071429  3.946078  3.185185   
69       3.92093  3.431818  3.259615  2.357143  3.071429  3.946078  3.185185   
606      2.50000  3.431818  3.259615  2.357143  3.071429  3.946078  2.500000   
248      3.92093  3.431818  3.259615  2.357143  3.071429  3.946078  3.185185   
Minimo   2.50000  3.431818  3.259615  2.357143  3.000000  3.946078  2.500000   

movieId  8       9         10      ...  193565  193567  193571  193573  \
userId                             ...                                   
479       2.875   3.125  3.496212  ...     3.5     

In [30]:
result_id = result
result_id

[120138, 2075, 26366, 128914, 100556, 26401, 162344, 4454, 162414, 100906]

In [31]:
def get_titles_from_ids(result_id, movies):
    titles = [movies.loc[movies['movieId'] == id_, 'title'].values[0] for id_ in result_id]
    return titles


# Esempio di utilizzo
titles = get_titles_from_ids(result_id, movies)
print(titles)


['PK (2014)', 'Mephisto (1981)', 'Harlan County U.S.A. (1976)', 'Tom Segura: Completely Normal (2014)', 'Act of Killing, The (2012)', 'Last Hurrah for Chivalry (Hao xia) (1979)', 'Tom Segura: Mostly Stories (2016)', 'More (1998)', 'Moonlight', 'Maniac Cop 2 (1990)']


##Su df Normalizzato

In [32]:
import pandas as pd

def order_items_by_least_misery_normalized(normalized_table):
    # Copia il DataFrame originale per non alterare i dati originali
    normalized_table_copy = normalized_table.copy()

    # Calcolare il voto minore per ogni ItemId tra le tre persone
    min_scores = normalized_table_copy.min(axis=0)

    # Trovare gli ItemId con lo stesso voto minimo
    equal_min_scores = min_scores[min_scores.duplicated(keep=False)].index.tolist()

    # Aggiungere la riga "Minimo" al DataFrame con il rating minore corrispondente ad ogni ItemId
    normalized_table_copy.loc['Minimo'] = min_scores

    # Stampa per visualizzare la tabella
    print(normalized_table_copy)

    # Creare la lista risultato
    result_list = []

 # Aggiungere gli ItemId alla lista in ordine decrescente
    for item in min_scores.index:
            result_list.append(item)

    # Ordinare la lista risultato in base all'ordinamento di sorted_items
    sorted_items = min_scores.sort_values(ascending=False)
    result_list.sort(key=lambda x: sorted_items.index.get_loc(x))

    # Ordinare la lista risultato in base all'ordinamento di sorted_items
    sorted_items = min_scores.sort_values(ascending=False)
    result_list.sort(key=lambda x: sorted_items.index.get_loc(x))

    return result_list[:10]

# Esempio di utilizzo

result_normalized = order_items_by_least_misery_normalized(normalized_table)
print(result_normalized)


movieId    1         2         3         4         5         6         7       \
userId                                                                          
479      0.658514  0.169402 -0.002801 -0.905273 -0.190988  0.683662 -0.077231   
276      0.733201  0.733201 -0.007183 -0.909656 -0.266799  0.679280  0.733201   
129      0.653691  0.164579 -0.007624 -0.910096 -0.195810  0.678840 -0.082054   
426      0.658248  1.237317 -0.003067 -0.905540 -0.191254  0.683396 -0.077497   
69       0.655263  0.166151 -0.006052 -0.908525 -0.194239  0.680411 -0.080482   
606     -0.770670  0.161148 -0.011055 -0.913527 -0.199241  0.675408 -0.770670   
248      0.658902  0.169790 -0.002413 -0.904885 -0.190600  0.684050 -0.076843   
Minimo  -0.770670  0.161148 -0.011055 -0.913527 -0.266799  0.675408 -0.770670   

movieId    8         9         10      ...    193565    193567    193571  \
userId                                 ...                                 
479     -0.387416 -0.137416  0.233796

In [33]:
result_id_normalized = result_normalized
result_id_normalized

[82744, 33649, 159811, 117531, 1349, 3951, 5328, 160644, 3942, 3941]

In [34]:
def get_titles_from_ids(result_id, movies):
    titles = [movies.loc[movies['movieId'] == id_, 'title'].values[0] for id_ in result_id]
    return titles


# Esempio di utilizzo
titles = get_titles_from_ids(result_id_normalized, movies)
print(titles)


['Faster (2010)', 'Saving Face (2004)', 'The Bremen Town Musicians (1969)', 'Watermark (2014)', 'Vampire in Venice (Nosferatu a Venezia) (Nosferatu in Venice) (1986)', 'Two Family House (2000)', 'Rain (2001)', 'Indignation (2016)', 'Sorority House Massacre II (1990)', 'Sorority House Massacre (1986)']


In [35]:
# Confronto se le due liste sono uguali
if result_id == result_id_normalized:
    print("Le liste sono uguali.")
else:
    print("Le liste non sono uguali.")

Le liste non sono uguali.


#Most Pleasure

##Su df non Normalizzato

Strategia inversa al Least Misery, quindi invece di creare una riga con il minimo, crea una riga con il massimo individuale.

In [36]:
import pandas as pd

def order_items_by_most_pleasure(random_rows):
    # Copia il DataFrame originale per non alterare i dati originali
    random_rows_copy = random_rows.copy()

    # Calcolare il voto massimo per ogni ItemId tra le tre persone
    max_scores = random_rows_copy.max(axis=0)

    # Trovare gli ItemId con lo stesso voto massimo
    equal_max_scores = max_scores[max_scores.duplicated(keep=False)].index.tolist()

    # Aggiungere la riga "Massimo" al DataFrame con il rating minore corrispondente ad ogni ItemId
    random_rows_copy.loc['Massimo'] = max_scores

    # Stampa per visualizzare la tabella
    print(random_rows_copy)

    # Creare la lista risultato
    result_list = []

    # Aggiungere gli ItemId alla lista in ordine decrescente con parentesi tonde per gli uguali
    for item in max_scores.index:
          result_list.append(item)

    # Ordinare la lista risultato in base all'ordinamento di sorted_items
    sorted_items = max_scores.sort_values(ascending=False)
    result_list.sort(key=lambda x: (sorted_items.index.get_loc(x)))

    return result_list[:10]

# Esempio di utilizzo

result = order_items_by_most_pleasure(random_rows)
print(result)


movieId   1         2         3         4         5         6         7       \
userId                                                                         
479      3.92093  3.431818  3.259615  2.357143  3.071429  3.946078  3.185185   
276      4.00000  4.000000  3.259615  2.357143  3.000000  3.946078  4.000000   
129      3.92093  3.431818  3.259615  2.357143  3.071429  3.946078  3.185185   
426      3.92093  4.500000  3.259615  2.357143  3.071429  3.946078  3.185185   
69       3.92093  3.431818  3.259615  2.357143  3.071429  3.946078  3.185185   
606      2.50000  3.431818  3.259615  2.357143  3.071429  3.946078  2.500000   
248      3.92093  3.431818  3.259615  2.357143  3.071429  3.946078  3.185185   
Massimo  4.00000  4.500000  3.259615  2.357143  3.071429  3.946078  4.000000   

movieId  8       9         10      ...  193565  193567  193571  193573  \
userId                             ...                                   
479       2.875   3.125  3.496212  ...     3.5     

In [37]:
result_id = result
result_id

[4495, 626, 113829, 593, 26147, 26169, 112512, 3939, 3940, 3941]

In [38]:
def get_titles_from_ids(result_id, movies):
    titles = [movies.loc[movies['movieId'] == id_, 'title'].values[0] for id_ in result_id]
    return titles


# Esempio di utilizzo
titles = get_titles_from_ids(result_id, movies)
print(titles)


['Crossing Delancey (1988)', 'Thin Line Between Love and Hate, A (1996)', 'One I Love, The (2014)', 'Silence of the Lambs, The (1991)', 'Thousand Clowns, A (1965)', 'Branded to Kill (Koroshi no rakuin) (1967)', 'Colourful (Karafuru) (2010)', 'Slumber Party Massacre II (1987)', 'Slumber Party Massacre III (1990)', 'Sorority House Massacre (1986)']


##Su df Normalizzato

In [39]:
import pandas as pd

def order_items_by_most_pleasure(normalized_table):
    # Copia il DataFrame originale per non alterare i dati originali
    normalized_table_copy = normalized_table.copy()

    # Calcolare il voto massimo per ogni ItemId tra le tre persone
    max_scores = normalized_table_copy.max(axis=0)

    # Trovare gli ItemId con lo stesso voto massimo
    equal_max_scores = max_scores[max_scores.duplicated(keep=False)].index.tolist()

    # Aggiungere la riga "Massimo" al DataFrame con il rating minore corrispondente ad ogni ItemId
    normalized_table_copy.loc['Massimo'] = max_scores

    # Stampa per visualizzare la tabella
    print(normalized_table_copy)

    # Creare la lista risultato
    result_list = []

    # Aggiungere gli ItemId alla lista in ordine decrescente con parentesi tonde per gli uguali
    for item in max_scores.index:
          result_list.append(item)

    # Ordinare la lista risultato in base all'ordinamento di sorted_items
    sorted_items = max_scores.sort_values(ascending=False)
    result_list.sort(key=lambda x: (sorted_items.index.get_loc(x)))

    return result_list[:10]

# Esempio di utilizzo

result_normalized = order_items_by_most_pleasure(normalized_table)
print(result_normalized)


movieId    1         2         3         4         5         6         7       \
userId                                                                          
479      0.658514  0.169402 -0.002801 -0.905273 -0.190988  0.683662 -0.077231   
276      0.733201  0.733201 -0.007183 -0.909656 -0.266799  0.679280  0.733201   
129      0.653691  0.164579 -0.007624 -0.910096 -0.195810  0.678840 -0.082054   
426      0.658248  1.237317 -0.003067 -0.905540 -0.191254  0.683396 -0.077497   
69       0.655263  0.166151 -0.006052 -0.908525 -0.194239  0.680411 -0.080482   
606     -0.770670  0.161148 -0.011055 -0.913527 -0.199241  0.675408 -0.770670   
248      0.658902  0.169790 -0.002413 -0.904885 -0.190600  0.684050 -0.076843   
Massimo  0.733201  1.237317 -0.002413 -0.904885 -0.190600  0.684050  0.733201   

movieId    8         9         10      ...    193565    193567    193571  \
userId                                 ...                                 
479     -0.387416 -0.137416  0.233796

In [40]:
result_id_normalized = result_normalized
result_id_normalized

[4135, 140133, 71268, 157775, 158027, 139640, 158398, 158882, 5537, 5513]

In [41]:
def get_titles_from_ids(result_id, movies):
    titles = [movies.loc[movies['movieId'] == id_, 'title'].values[0] for id_ in result_id]
    return titles


# Esempio di utilizzo
titles = get_titles_from_ids(result_id_normalized, movies)
print(titles)


['Monster Squad, The (1987)', 'Hollywood Chainsaw Hookers (1988)', "Tyler Perry's I Can Do Bad All by Myself (2009)", 'Tenchi MuyÃ´! In Love (1996)', 'SORI: Voice from the Heart (2016)', 'Ooops! Noah is Gone... (2015)', 'World of Glory (1991)', 'All Yours (2016)', 'Satin Rouge (2002)', 'Martin Lawrence Live: Runteldat (2002)']


In [42]:
# Confronto se le due liste sono uguali
if result_id == result_id_normalized:
    print("Le liste sono uguali.")
else:
    print("Le liste non sono uguali.")

Le liste non sono uguali.


#Utilitarian Strategy

Sono utilizzati valori di utilità invece di usare solamente informazioni che derivano dai ranking. Per esempio la **somma**, maggiore è la somma dei voti dei singoli item e prima troveremo l’opzione nella classifica.Il ranking creato in questo modo sarà esattamente identico al ranking in caso si usasse la “Avarage Strategy”. Oltre alla somma i vari rating si possono **moltiplicare**, l’ordine della lista sia diversa utilizzando la somma.

##Additive (Non Normalizated)

In [43]:
import pandas as pd

def order_items_by_additive(random_rows):
    # Copia il DataFrame originale per non alterare i dati originali
    random_rows_copy = random_rows.copy()

    # Calcolare la somma dei voti ricevuti per ciascun ItemId
    votes_sum = random_rows_copy.sum(axis=0)

    # Aggiungere la colonna "somma" al DataFrame con la somma dei voti per ciascun ItemId
    random_rows_copy.loc['somma'] = votes_sum

    # Stampa per visualizzare la tabella
    print(random_rows_copy)

    # Trovare item con la stessa somma
    equal_sums = votes_sum[votes_sum.duplicated(keep=False)].index.tolist()

    # Ordinare gli ItemId in base alla somma dei voti in ordine decrescente
    sorted_items = votes_sum.sort_values(ascending=False).index

    # Creare la lista risultato
    result_list = []

    # Aggiungere gli ItemId alla lista in base alla somma dei voti in ordine decrescente con parentesi tonde per gli uguali
    for item in sorted_items:
            result_list.append(item)

    return result_list[:10]

# Esempio di utilizzo
result = order_items_by_additive(random_rows)
print(result)


movieId     1          2          3          4          5          6       \
userId                                                                      
479       3.920930   3.431818   3.259615   2.357143   3.071429   3.946078   
276       4.000000   4.000000   3.259615   2.357143   3.000000   3.946078   
129       3.920930   3.431818   3.259615   2.357143   3.071429   3.946078   
426       3.920930   4.500000   3.259615   2.357143   3.071429   3.946078   
69        3.920930   3.431818   3.259615   2.357143   3.071429   3.946078   
606       2.500000   3.431818   3.259615   2.357143   3.071429   3.946078   
248       3.920930   3.431818   3.259615   2.357143   3.071429   3.946078   
somma    26.104651  25.659091  22.817308  16.500000  21.428571  27.622549   

movieId     7       8       9          10      ...  193565  193567  193571  \
userId                                         ...                           
479       3.185185   2.875   3.125   3.496212  ...     3.5     3.0     4.

In [44]:
result_id = result
result_id

[2196, 64501, 138966, 163925, 138835, 64499, 83969, 138632, 27523, 164367]

In [45]:
def get_titles_from_ids(result_id, movies):
    titles = [movies.loc[movies['movieId'] == id_, 'title'].values[0] for id_ in result_id]
    return titles


# Esempio di utilizzo
titles = get_titles_from_ids(result_id, movies)
print(titles)


['Knock Off (1998)', 'Che: Part Two (2008)', 'Nasu: Summer in Andalusia (2003)', 'Wings, Legs and Tails (1986)', 'Return to Treasure Island (1988)', 'Che: Part One (2008)', 'Down Argentine Way (1940)', 'Tokyo Tribe (2014)', 'My Sassy Girl (Yeopgijeogin geunyeo) (2001)', 'The Girl with All the Gifts (2016)']


##Additive (Normalizated)

In [46]:
import pandas as pd

def order_items_by_additive(normalized_table):
    # Copia il DataFrame originale per non alterare i dati originali
    normalized_table_copy = normalized_table.copy()

    # Calcolare la somma dei voti ricevuti per ciascun ItemId
    votes_sum = normalized_table_copy.sum(axis=0)

    # Aggiungere la colonna "somma" al DataFrame con la somma dei voti per ciascun ItemId
    normalized_table_copy.loc['somma'] = votes_sum

    # Stampa per visualizzare la tabella
    print(normalized_table_copy)

    # Trovare item con la stessa somma
    equal_sums = votes_sum[votes_sum.duplicated(keep=False)].index.tolist()

    # Ordinare gli ItemId in base alla somma dei voti in ordine decrescente
    sorted_items = votes_sum.sort_values(ascending=False).index

    # Creare la lista risultato
    result_list = []

    # Aggiungere gli ItemId alla lista in base alla somma dei voti in ordine decrescente con parentesi tonde per gli uguali
    for item in sorted_items:
            result_list.append(item)

    return result_list[:10]

# Esempio di utilizzo
result_normalized = order_items_by_additive(normalized_table)
print(result_normalized)


movieId    1         2         3         4         5         6         7       \
userId                                                                          
479      0.658514  0.169402 -0.002801 -0.905273 -0.190988  0.683662 -0.077231   
276      0.733201  0.733201 -0.007183 -0.909656 -0.266799  0.679280  0.733201   
129      0.653691  0.164579 -0.007624 -0.910096 -0.195810  0.678840 -0.082054   
426      0.658248  1.237317 -0.003067 -0.905540 -0.191254  0.683396 -0.077497   
69       0.655263  0.166151 -0.006052 -0.908525 -0.194239  0.680411 -0.080482   
606     -0.770670  0.161148 -0.011055 -0.913527 -0.199241  0.675408 -0.770670   
248      0.658902  0.169790 -0.002413 -0.904885 -0.190600  0.684050 -0.076843   
somma    3.247149  2.801589 -0.040194 -6.357502 -1.428931  4.765047 -0.431576   

movieId    8         9         10      ...    193565    193567    193571  \
userId                                 ...                                 
479     -0.387416 -0.137416  0.233796

In [47]:
result_id_normalized = result_normalized
result_id_normalized

[2196, 64501, 138966, 163925, 138835, 64499, 83969, 138632, 27523, 164367]

In [48]:
def get_titles_from_ids(result_id, movies):
    titles = [movies.loc[movies['movieId'] == id_, 'title'].values[0] for id_ in result_id]
    return titles


# Esempio di utilizzo
titles = get_titles_from_ids(result_id_normalized, movies)
print(titles)


['Knock Off (1998)', 'Che: Part Two (2008)', 'Nasu: Summer in Andalusia (2003)', 'Wings, Legs and Tails (1986)', 'Return to Treasure Island (1988)', 'Che: Part One (2008)', 'Down Argentine Way (1940)', 'Tokyo Tribe (2014)', 'My Sassy Girl (Yeopgijeogin geunyeo) (2001)', 'The Girl with All the Gifts (2016)']


In [49]:
# Confronto se le due liste sono uguali
if result_id == result_id_normalized:
    print("Le liste sono uguali.")
else:
    print("Le liste non sono uguali.")

Le liste sono uguali.


## Multiplicative (Non Normalizated)

In [50]:
import pandas as pd

def order_items_by_product(random_rows):
    # Copia il DataFrame originale per non alterare i dati originali
    random_rows_copy = random_rows.copy()

    # Calcolare il prodotto dei voti ricevuti per ciascun ItemId
    votes_product = random_rows_copy.prod(axis=0)

    # Aggiungere la colonna "prodotto" al DataFrame con il prodotto dei voti per ciascun ItemId
    random_rows_copy.loc['prodotto'] = votes_product

    # Stampa per visualizzare la tabella
    print(random_rows_copy)

    # Trovare item con lo stesso prodotto
    equal_products = votes_product[votes_product.duplicated(keep=False)].index.tolist()

    # Ordinare gli ItemId in base al prodotto dei voti in ordine decrescente
    sorted_items = votes_product.sort_values(ascending=False).index

    # Creare la lista risultato
    result_list = []

    # Aggiungere gli ItemId alla lista in base al prodotto dei voti in ordine decrescente con parentesi tonde per gli uguali
    for item in sorted_items:
            result_list.append(item)

    return result_list[:10]

# Esempio di utilizzo

result = order_items_by_product(random_rows)
print(result)


movieId        1            2            3           4            5       \
userId                                                                     
479          3.920930     3.431818     3.259615    2.357143     3.071429   
276          4.000000     4.000000     3.259615    2.357143     3.000000   
129          3.920930     3.431818     3.259615    2.357143     3.071429   
426          3.920930     4.500000     3.259615    2.357143     3.071429   
69           3.920930     3.431818     3.259615    2.357143     3.071429   
606          2.500000     3.431818     3.259615    2.357143     3.071429   
248          3.920930     3.431818     3.259615    2.357143     3.071429   
prodotto  9267.136789  8568.284122  3909.889758  404.297755  2518.626532   

movieId         6            7           8            9            10      \
userId                                                                      
479           3.946078     3.185185     2.87500     3.125000     3.496212   
276     

In [51]:
result_id = result
result_id

[95843, 6835, 113829, 77846, 26401, 3637, 5607, 78836, 26366, 5537]

In [52]:
def get_titles_from_ids(result_id, movies):
    titles = [movies.loc[movies['movieId'] == id_, 'title'].values[0] for id_ in result_id]
    return titles


# Esempio di utilizzo
titles = get_titles_from_ids(result_id, movies)
print(titles)


['Goodbye Charlie (1964)', 'Alien Contamination (1980)', 'One I Love, The (2014)', '12 Angry Men (1997)', 'Last Hurrah for Chivalry (Hao xia) (1979)', 'Vagabond (Sans toit ni loi) (1985)', 'Son of the Bride (Hijo de la novia, El) (2001)', 'Enter the Void (2009)', 'Harlan County U.S.A. (1976)', 'Satin Rouge (2002)']


##Multiplicative (Normalizated)

In [53]:
import pandas as pd

def order_items_by_product(normalized_table):
    # Copia il DataFrame originale per non alterare i dati originali
    normalized_table_copy = normalized_table.copy()

    # Calcolare il prodotto dei voti ricevuti per ciascun ItemId
    votes_product = normalized_table_copy.prod(axis=0)

    # Aggiungere la colonna "prodotto" al DataFrame con il prodotto dei voti per ciascun ItemId
    normalized_table_copy.loc['prodotto'] = votes_product

    # Stampa per visualizzare la tabella
    print(normalized_table_copy)

    # Trovare item con lo stesso prodotto
    equal_products = votes_product[votes_product.duplicated(keep=False)].index.tolist()

    # Ordinare gli ItemId in base al prodotto dei voti in ordine decrescente
    sorted_items = votes_product.sort_values(ascending=False).index

    # Creare la lista risultato
    result_list = []

    # Aggiungere gli ItemId alla lista in base al prodotto dei voti in ordine decrescente con parentesi tonde per gli uguali
    for item in sorted_items:
            result_list.append(item)

    return result_list[:10]

# Esempio di utilizzo

result_normalized = order_items_by_product(normalized_table)
print(result_normalized)


movieId     1         2             3         4         5         6       \
userId                                                                     
479       0.658514  0.169402 -2.800811e-03 -0.905273 -0.190988  0.683662   
276       0.733201  0.733201 -7.183121e-03 -0.909656 -0.266799  0.679280   
129       0.653691  0.164579 -7.623537e-03 -0.910096 -0.195810  0.678840   
426       0.658248  1.237317 -3.067128e-03 -0.905540 -0.191254  0.683396   
69        0.655263  0.166151 -6.052192e-03 -0.908525 -0.194239  0.680411   
606      -0.770670  0.161148 -1.105464e-02 -0.913527 -0.199241  0.675408   
248       0.658902  0.169790 -2.412813e-03 -0.904885 -0.190600  0.684050   
prodotto -0.069128  0.000115 -7.593926e-17 -0.509687 -0.000014  0.067726   

movieId     7         8         9         10      ...    193565    193567  \
userId                                            ...                       
479      -0.077231 -0.387416 -0.137416  0.233796  ...  0.237584 -0.262416   
276     

In [54]:
result_id_normalized = result_normalized
result_id_normalized

[27704, 44943, 495, 27320, 102084, 100906, 64499, 157775, 158027, 64501]

In [55]:
def get_titles_from_ids(result_id, movies):
    titles = [movies.loc[movies['movieId'] == id_, 'title'].values[0] for id_ in result_id]
    return titles


# Esempio di utilizzo
titles = get_titles_from_ids(result_id_normalized, movies)
print(titles)

['Battle Royale 2: Requiem (Batoru rowaiaru II: Chinkonka) (2003)', '9/11 (2002)', 'In the Realm of the Senses (Ai no corrida) (1976)', 'Nine Lives of Tomas Katz, The (2000)', 'Justice League: Doom (2012) ', 'Maniac Cop 2 (1990)', 'Che: Part One (2008)', 'Tenchi MuyÃ´! In Love (1996)', 'SORI: Voice from the Heart (2016)', 'Che: Part Two (2008)']


In [56]:
# Confronto se le due liste sono uguali
if result_id == result_id_normalized:
    print("Le liste sono uguali.")
else:
    print("Le liste non sono uguali.")

Le liste non sono uguali.


#Borda Count

Strategia in cui i punti sono assegnati ad ogni opzione secondo la posizione che occupano nella lista delle preferenze di ogni singolo individuo. Quindi l’item con voto minore avrà punteggio 0. Nel caso in cui ci siano diverse  alternative con lo stesso rating, avviene una distribuzione dei punti.

##Su df non Normalizzato


In [57]:
import pandas as pd

def create_borda_preference_table(data):
    # Creare una nuova tabella per le preferenze di Borda Count
    borda_table = pd.DataFrame(index=data.index, columns=data.columns[:-1])

    # Popolare la tabella con le preferenze di Borda Count
    for voter in data.columns[:-1]:
        ranked_items = data.sort_values(by=voter, ascending=False).index.tolist()
        n_ranks = len(ranked_items)

        # Calcolare i punti per ogni item considerando le posizioni
        points_dict = {}
        for i, item in enumerate(ranked_items):
            points_dict[item] = i  # Assegna il punteggio basato sulla posizione

        # Assegna i punti alla colonna corrispondente nella tabella
        borda_table[voter] = borda_table.index.to_series().map(points_dict)

    # Aggiungere la riga "Group" con la somma dei ranking
    borda_table['Group'] = borda_table.sum(axis=1)

    # Stampa
    print(borda_table)

    # Ordinare la tabella in base alla colonna "Group" in modo decrescente
    sorted_borda_table = borda_table.sort_values(by='Group', ascending=False)

      # Creare la lista ordinata con elementi in parentesi tonde in caso di valori uguali
    sorted_items = []
    for idx, row in sorted_borda_table.iterrows():
        item_str = idx
        sorted_items.append(item_str)

    return sorted_items[:10]


# Esempio di utilizzo
result = create_borda_preference_table(random_rows.T)
print(result)


userId    479   276   129   426    69   606  Group
movieId                                           
1        2461  1553  2470  2434  2430  7845  19193
2        4815  1954  4858   417  4825  4908  21777
3        5304  5345  5348  5317  5334  5330  31978
4        8310  8340  8337  8325  8339  8323  49974
5        5906  6116  5972  5934  5970  5874  35772
...       ...   ...   ...   ...   ...   ...    ...
193581   1913  1759  1598  1895  2116  2412  11693
193583   3790  3984  4163  3762  3902  3846  23447
193585   3825  3985  4162  3829  3903  3847  23551
193587   3816  3989  4161  3818  3904  3848  23536
193609   1667  2156  2317  1656  1649  1819  11264

[9724 rows x 7 columns]
[3933, 76030, 54768, 8772, 6514, 110603, 110773, 173307, 5356, 111785]


In [58]:
result_id = result
result_id

[3933, 76030, 54768, 8772, 6514, 110603, 110773, 173307, 5356, 111785]

In [59]:
def get_titles_from_ids(result_id, movies):
    titles = [movies.loc[movies['movieId'] == id_, 'title'].values[0] for id_ in result_id]
    return titles


# Esempio di utilizzo
titles = get_titles_from_ids(result_id, movies)
print(titles)


['Killer Shrews, The (1959)', 'Case 39 (2009)', 'Daddy Day Camp (2007)', 'Spy Who Came in from the Cold, The (1965)', 'Ring of Terror (1962)', "God's Not Dead (2014)", 'Haunted House 2, A (2014)', 'The Gracefield Incident (2015)', 'Giant Spider Invasion, The (1975)', 'Midnight Chronicles (2009)']


## Su df normalizzato

In [ ]:
import pandas as pd

def create_borda_preference_table(data):
    # Creare una nuova tabella per le preferenze di Borda Count
    borda_table = pd.DataFrame(index=data.index, columns=data.columns[:-1])

    # Popolare la tabella con le preferenze di Borda Count
    for voter in data.columns[:-1]:
        ranked_items = data.sort_values(by=voter, ascending=False).index.tolist()
        n_ranks = len(ranked_items)

        # Calcolare i punti per ogni item considerando le posizioni
        points_dict = {}
        for i, item in enumerate(ranked_items):
            points_dict[item] = i  # Assegna il punteggio basato sulla posizione

        # Assegna i punti alla colonna corrispondente nella tabella
        borda_table[voter] = borda_table.index.to_series().map(points_dict)

    # Aggiungere la riga "Group" con la somma dei ranking
    borda_table['Group'] = borda_table.sum(axis=1)

    # Stampa
    print(borda_table)

    # Ordinare la tabella in base alla colonna "Group" in modo decrescente
    sorted_borda_table = borda_table.sort_values(by='Group', ascending=False)

      # Creare la lista ordinata con elementi in parentesi tonde in caso di valori uguali
    sorted_items = []
    for idx, row in sorted_borda_table.iterrows():
        item_str = idx
        sorted_items.append(item_str)

    return sorted_items[:10]


# Esempio di utilizzo
result = create_borda_preference_table(normalized_table.T)
print(result)


userId    479   276   129   426    69   606  Group
movieId                                           
1        2461  1553  2470  2434  2430  7845  19193
2        4815  1954  4858   417  4825  4908  21777
3        5304  5345  5348  5317  5334  5330  31978
4        8310  8340  8337  8325  8339  8323  49974
5        5906  6116  5972  5934  5970  5874  35772
...       ...   ...   ...   ...   ...   ...    ...
193581   1913  1759  1598  1895  2116  2412  11693
193583   3790  3984  4163  3762  3902  3846  23447
193585   3825  3985  4162  3829  3903  3847  23551
193587   3816  3989  4161  3818  3904  3848  23536
193609   1667  2156  2317  1656  1649  1819  11264

[9724 rows x 7 columns]


In [ ]:
result_id_normalized = result
result_id_normalized

In [ ]:
def get_titles_from_ids(result_id, movies):
    titles = [movies.loc[movies['movieId'] == id_, 'title'].values[0] for id_ in result_id]
    return titles


# Esempio di utilizzo
titles = get_titles_from_ids(result_id_normalized, movies)
print(titles)


In [ ]:
# Confronto se le due liste sono uguali
if result_id == result_id_normalized:
    print("Le liste sono uguali.")
else:
    print("Le liste non sono uguali.")

#Copeland rule

E’ una foto di voto per maggioranza, si costruisce una matrice item x item in cui viene assegnato segno in base se A batte le alternative B,C,D… Sulla diagonale maggiore ci saranno tutti 0 e appariranno 0 sparsi nella matrice quando le due alternative hanno pari voto. La strategia per df in cui ci sono moltissimi item come in questo caso ha costo computazionale elevato. Per questo ho semplificato il problema eseguendo la strategia su una matrice con item ridotti

##Su df non normalizzato

In [ ]:
import pandas as pd

def copeland_rule_table(data):
    # Creare una nuova tabella per la regola di Copeland
    copeland_table = pd.DataFrame(index=data.index, columns=data.index)

    # Popolare la tabella con la regola di Copeland
    for row in data.index:
        for col in data.index:
            if row == col:
                copeland_table.at[row, col] = 0
            else:
                wins = 0
                losses = 0
                for voter in data.columns:
                    if data.at[row, voter] < data.at[col, voter]:
                        wins += 1
                    elif data.at[row, voter] > data.at[col, voter]:
                        losses += 1
                copeland_table.at[row, col] = wins - losses

                # Modifica per ottenere solo il segno
                if copeland_table.at[row, col] > 0:
                    copeland_table.at[row, col] = 1
                elif copeland_table.at[row, col] < 0:
                    copeland_table.at[row, col] = -1

    # Aggiungi una riga "Index" con la somma dei + e dei -
    copeland_table.loc["Index"] = copeland_table.apply(lambda x: sum(x), axis=0)

    # Calcola la lista degli item ordinati
    sorted_items = copeland_table.loc["Index"].sort_values(ascending=False).index.tolist()

    # Modifica per includere item con lo stesso indice in parentesi tonde
    final_sorted_items = []
    current_group = []
    current_value = None

    # Stampa la copeland_table
    print(copeland_table)

    for item in sorted_items:
        value = copeland_table.loc["Index", item]
        final_sorted_items.append(item)
        current_value = value

    return final_sorted_items [:10]

# Esempio di utilizzo con la tabella data
copeland_result = copeland_rule_table(random_rows.iloc[:,:150].T)
print(copeland_result)


In [ ]:
result_id = copeland_result
result_id

In [ ]:
def get_titles_from_ids(result_id, movies):
    titles = [movies.loc[movies['movieId'] == id_, 'title'].values[0] for id_ in result_id]
    return titles


# Esempio di utilizzo
titles = get_titles_from_ids(result_id, movies)
print(titles)


##Su df normalizzato

In [ ]:
import pandas as pd

def copeland_rule_table(data):
    # Creare una nuova tabella per la regola di Copeland
    copeland_table = pd.DataFrame(index=data.index, columns=data.index)

    # Popolare la tabella con la regola di Copeland
    for row in data.index:
        for col in data.index:
            if row == col:
                copeland_table.at[row, col] = 0
            else:
                wins = 0
                losses = 0
                for voter in data.columns:
                    if data.at[row, voter] < data.at[col, voter]:
                        wins += 1
                    elif data.at[row, voter] > data.at[col, voter]:
                        losses += 1
                copeland_table.at[row, col] = wins - losses

                # Modifica per ottenere solo il segno
                if copeland_table.at[row, col] > 0:
                    copeland_table.at[row, col] = 1
                elif copeland_table.at[row, col] < 0:
                    copeland_table.at[row, col] = -1

    # Aggiungi una riga "Index" con la somma dei + e dei -
    copeland_table.loc["Index"] = copeland_table.apply(lambda x: sum(x), axis=0)

    # Calcola la lista degli item ordinati
    sorted_items = copeland_table.loc["Index"].sort_values(ascending=False).index.tolist()

    # Modifica per includere item con lo stesso indice in parentesi tonde
    final_sorted_items = []
    current_group = []
    current_value = None

    # Stampa la copeland_table
    print(copeland_table)

    for item in sorted_items:
        value = copeland_table.loc["Index", item]
        final_sorted_items.append(item)
        current_value = value

    return final_sorted_items [:10]

# Esempio di utilizzo con la tabella data
copeland_result = copeland_rule_table(normalized_table.iloc[:,:150].T)
print(copeland_result)


In [ ]:
result_id_normalized = copeland_result
result_id_normalized

In [ ]:
def get_titles_from_ids(result_id, movies):
    titles = [movies.loc[movies['movieId'] == id_, 'title'].values[0] for id_ in result_id]
    return titles


# Esempio di utilizzo
titles = get_titles_from_ids(result_id_normalized, movies)
print(titles)


In [ ]:
# Confronto se le due liste sono uguali
if result_id == result_id_normalized:
    print("Le liste sono uguali.")
else:
    print("Le liste non sono uguali.")

#Approval Voting

I votanti hanno la possibilità di votare più alternative in questo modo si ha la promozione di più alternative che superano una certa soglia, andando ad escludere le alternative maggiormente non piaciute.

##Su df non normalizzato

In [ ]:
import pandas as pd

def approval_voting_strategy(normalized_table, threshold):
    # Creare una copia del DataFrame originale per non alterare i dati originali
    approval_voting_table = normalized_table.copy()

    # Applica la soglia alle valutazioni
    approval_voting_table[approval_voting_table <= threshold] = 0
    approval_voting_table[approval_voting_table > threshold] = 1

     # Calcola la somma degli Approval Voting per ogni item
    approval_voting_table.loc['Somma'] = approval_voting_table.sum(axis=0)

    # Stampa per visualizzare la tabella
    print(approval_voting_table)

     # Calcola la lista degli item ordinati in base alla somma
    sorted_items = approval_voting_table.loc['Somma'].sort_values(ascending=False).index.tolist()

    # Modifica per includere item con lo stesso indice senza parentesi tonde
    final_sorted_items = sorted_items

    return final_sorted_items[:10]

# Esempio di utilizzo con la tabella data
threshold = 4
approval_result = approval_voting_strategy(random_rows, threshold)
print(approval_result)


In [ ]:
result_id = approval_result
result_id

In [ ]:
def get_titles_from_ids(result_id, movies):
    titles = [movies.loc[movies['movieId'] == id_, 'title'].values[0] for id_ in result_id]
    return titles


# Esempio di utilizzo
titles = get_titles_from_ids(result_id, movies)
print(titles)

##Su df normalizzato

In [ ]:
import pandas as pd

def approval_voting_strategy(normalized_table, threshold):
    # Creare una copia del DataFrame originale per non alterare i dati originali
    approval_voting_table = normalized_table.copy()

    # Applica la soglia alle valutazioni
    approval_voting_table[approval_voting_table <= threshold] = 0
    approval_voting_table[approval_voting_table > threshold] = 1

     # Calcola la somma degli Approval Voting per ogni item
    approval_voting_table.loc['Somma'] = approval_voting_table.sum(axis=0)

    # Stampa per visualizzare la tabella
    print(approval_voting_table)

     # Calcola la lista degli item ordinati in base alla somma
    sorted_items = approval_voting_table.loc['Somma'].sort_values(ascending=False).index.tolist()

    # Modifica per includere item con lo stesso indice senza parentesi tonde
    final_sorted_items = sorted_items

    return final_sorted_items[:10]

# Esempio di utilizzo con la tabella data
threshold = 0.5
approval_result = approval_voting_strategy(normalized_table, threshold)
print(approval_result)


In [ ]:
result_id_normalized = approval_result
result_id_normalized

In [ ]:
def get_titles_from_ids(result_id, movies):
    titles = [movies.loc[movies['movieId'] == id_, 'title'].values[0] for id_ in result_id]
    return titles


# Esempio di utilizzo
titles = get_titles_from_ids(result_id_normalized, movies)
print(titles)

In [ ]:
# Confronto se le due liste sono uguali
if result_id == result_id_normalized:
    print("Le liste sono uguali.")
else:
    print("Le liste non sono uguali.")

#DictatorShip

Vengono presi in considerazioni le preferenze della persona più rispettata. Alla base di questa strategia è che ogni gruppo può essere dominato da una persona. Per semplicità, non avendo dati anagrafici o criteri su cui determinare la persona più rispettata, ho ipotizzato che sia la persona con id corrispondente alla prima riga

##Su df non normalizzato

In [ ]:
import pandas as pd

def dictatorship_strategy(ratings, respected_person_index=1):
    # Scegli la persona più rispettata in base all'indice fornito
    most_respected_person = ratings.columns[respected_person_index]

    # Crea una nuova riga "Group" contenente i voti della persona più rispettata
    ratings.loc['Group'] = ratings.iloc[0]

    # Ordina gli item in base ai voti in ordine decrescente
    sorted_items = ratings.loc['Group'].sort_values(ascending=False).index.tolist()

    # Stampa per visualizzare la tabella
    print(ratings)

    # Creare la lista risultato
    result_list = []

    # Aggiungere gli ItemId alla lista in base al prodotto dei voti in ordine decrescente con parentesi tonde per gli uguali
    for item in sorted_items:
            result_list.append(item)

    return result_list[:10]

# Esempio di utilizzo
# Specifica l'indice della persona più rispettata (indice 0-based, quindi scelgo il primo della lista)
result = dictatorship_strategy(random_rows, respected_person_index=0)
print(result)


In [ ]:
result_id = result
result_id

In [ ]:
def get_titles_from_ids(result_id, movies):
    titles = [movies.loc[movies['movieId'] == id_, 'title'].values[0] for id_ in result_id]
    return titles


# Esempio di utilizzo
titles = get_titles_from_ids(result_id, movies)
print(titles)

## Su df normalizzato

In [ ]:
import pandas as pd

def dictatorship_strategy(ratings, respected_person_index=1):
    # Scegli la persona più rispettata in base all'indice fornito
    most_respected_person = ratings.columns[respected_person_index]

    # Crea una nuova riga "Group" contenente i voti della persona più rispettata
    ratings.loc['Group'] = ratings.iloc[0]

    # Ordina gli item in base ai voti in ordine decrescente
    sorted_items = ratings.loc['Group'].sort_values(ascending=False).index.tolist()

    # Stampa per visualizzare la tabella
    print(ratings)

    # Creare la lista risultato
    result_list = []

    # Aggiungere gli ItemId alla lista in base al prodotto dei voti in ordine decrescente con parentesi tonde per gli uguali
    for item in sorted_items:
            result_list.append(item)

    return result_list[:10]

# Esempio di utilizzo
# Specifica l'indice della persona più rispettata (indice 0-based, quindi scelgo il primo della lista)
result = dictatorship_strategy(normalized_table, respected_person_index=0)
print(result)


In [ ]:
result_id_normalized = result
result_id_normalized

In [ ]:
def get_titles_from_ids(result_id, movies):
    titles = [movies.loc[movies['movieId'] == id_, 'title'].values[0] for id_ in result_id]
    return titles


# Esempio di utilizzo
titles = get_titles_from_ids(result_id_normalized, movies)
print(titles)

In [ ]:
# Confronto se le due liste sono uguali
if result_id == result_id_normalized:
    print("Le liste sono uguali.")
else:
    print("Le liste non sono uguali.")

#Fairness Strategy



Sono selezionati i top item da ogni individuo, quando ci sono item votati uguali, vengono prese in considerazioni altre opzioni, in questo caso per semplicità ho considerato il primo item trovato. Alla base di questa idea è che non è tanto male guardare qualcosa che odi finchè puoi vedere qualcosa che ami successivamente. “Chi sceglie ultimo al secondo round sceglie prima”.

##Su df non normalizzato

In [ ]:
import pandas as pd

def fairness_strategy(ratings):
    # Crea una copia del DataFrame originale per evitare di modificare i dati originali
    ratings_copy = ratings.copy()

    # Crea una lista vuota per memorizzare l'ordine finale degli elementi
    result_list = []

    # Stampa per visualizzare la tabella
    print(ratings_copy)

    # Itera finché tutti gli elementi non sono selezionati o ratings_copy è vuoto
    while not ratings_copy.empty:
        # Primo giro: seleziona gli elementi nell'ordine da John a Mary
        for person in ratings_copy.columns:
            # Verifica se ci sono ancora elementi da selezionare
            if not ratings_copy.empty:
                # Seleziona l'elemento con il rating più alto per la persona corrente
                top_item = ratings_copy[person].idxmax()

                # Aggiungi l'elemento selezionato alla lista dei risultati
                result_list.append(top_item)

                # Rimuovi l'elemento selezionato dal DataFrame
                ratings_copy = ratings_copy.drop(index=top_item)

        # Secondo giro: seleziona gli elementi nell'ordine inverso
        for person in ratings_copy.columns[::-1]:
            # Verifica se ci sono ancora elementi da selezionare
            if not ratings_copy.empty:
                # Seleziona l'elemento con il rating più alto per la persona corrente
                top_item = ratings_copy[person].idxmax()

                # Aggiungi l'elemento selezionato alla lista dei risultati
                result_list.append(top_item)

                # Rimuovi l'elemento selezionato dal DataFrame
                ratings_copy = ratings_copy.drop(index=top_item)

    return result_list[:10]

# Applicazione Fairness Strategy
result = fairness_strategy(random_rows.T)
print(result)


In [ ]:
result_id = result
result_id

In [ ]:
def get_titles_from_ids(result_id, movies):
    titles = [movies.loc[movies['movieId'] == id_, 'title'].values[0] for id_ in result_id]
    return titles


# Esempio di utilizzo
titles = get_titles_from_ids(result_id, movies)
print(titles)

##Su df normalizzato

In [ ]:
import pandas as pd

def fairness_strategy(ratings):
    # Crea una copia del DataFrame originale per evitare di modificare i dati originali
    ratings_copy = ratings.copy()

    # Crea una lista vuota per memorizzare l'ordine finale degli elementi
    result_list = []

    # Stampa per visualizzare la tabella
    print(ratings_copy)

    # Itera finché tutti gli elementi non sono selezionati o ratings_copy è vuoto
    while not ratings_copy.empty:
        # Primo giro: seleziona gli elementi nell'ordine da John a Mary
        for person in ratings_copy.columns:
            # Verifica se ci sono ancora elementi da selezionare
            if not ratings_copy.empty:
                # Seleziona l'elemento con il rating più alto per la persona corrente
                top_item = ratings_copy[person].idxmax()

                # Aggiungi l'elemento selezionato alla lista dei risultati
                result_list.append(top_item)

                # Rimuovi l'elemento selezionato dal DataFrame
                ratings_copy = ratings_copy.drop(index=top_item)

        # Secondo giro: seleziona gli elementi nell'ordine inverso
        for person in ratings_copy.columns[::-1]:
            # Verifica se ci sono ancora elementi da selezionare
            if not ratings_copy.empty:
                # Seleziona l'elemento con il rating più alto per la persona corrente
                top_item = ratings_copy[person].idxmax()

                # Aggiungi l'elemento selezionato alla lista dei risultati
                result_list.append(top_item)

                # Rimuovi l'elemento selezionato dal DataFrame
                ratings_copy = ratings_copy.drop(index=top_item)

    return result_list[:10]

# Applicazione Fairness Strategy
result = fairness_strategy(normalized_table.T)
print(result)


In [ ]:
result_id_normalized = result
result_id_normalized

In [ ]:
def get_titles_from_ids(result_id, movies):
    titles = [movies.loc[movies['movieId'] == id_, 'title'].values[0] for id_ in result_id]
    return titles


# Esempio di utilizzo
titles = get_titles_from_ids(result_id_normalized, movies)
print(titles)

In [ ]:
# Confronto se le due liste sono uguali
if result_id == result_id_normalized:
    print("Le liste sono uguali.")
else:
    print("Le liste non sono uguali.")

#Plurality Voting

L’alternativa con il il maggior numero di voti vince. Bisogna valutare il caso in cui il voto finisca in parità senza ottenere alcun vincitore. La strategia è quella di far votare per tutti gli item e dare la possibilità di dare un voto massimo a più item

##Su df non normalizzato



In [ ]:
import pandas as pd

def strategia_plurality_voting(ratings):
    # Crea una copia del DataFrame originale per evitare di modificare i dati originali
    ratings_copia = ratings.copy()

    #Stampa
    print(ratings_copia)

    # Crea una lista vuota per memorizzare l'ordine finale degli elementi
    lista_risultati = []

    # Itera finché tutti gli elementi non sono stati selezionati o ratings_copia è vuoto
    while not ratings_copia.empty:
        # Conta il numero di voti per ciascun elemento
        voti_per_elemento = ratings_copia.apply(lambda x: (x == x.max()).astype(int))

        # Somma i voti per ottenere il numero totale di voti per ciascun elemento
        voti_totali = voti_per_elemento.sum(axis=1)

        # Seleziona l'elemento con più voti
        elemento_piu_votato = voti_totali.idxmax()

        # Aggiungi l'elemento più votato alla lista dei risultati
        lista_risultati.append(elemento_piu_votato)

        # Rimuovi l'elemento selezionato dal DataFrame utilizzando l'indice
        ratings_copia = ratings_copia.drop(index=elemento_piu_votato)

    return lista_risultati[:10]

# Applica la strategia di Plurality Voting
risultato = strategia_plurality_voting(random_rows.T)
print(risultato)


In [ ]:
result_id = risultato
result_id

In [ ]:
def get_titles_from_ids(result_id, movies):
    titles = [movies.loc[movies['movieId'] == id_, 'title'].values[0] for id_ in result_id]
    return titles


# Esempio di utilizzo
titles = get_titles_from_ids(result_id, movies)
print(titles)

##Su df non normalizzato

In [ ]:
import pandas as pd

def strategia_plurality_voting(ratings):
    # Crea una copia del DataFrame originale per evitare di modificare i dati originali
    ratings_copia = ratings.copy()

    #Stampa
    print(ratings_copia)

    # Crea una lista vuota per memorizzare l'ordine finale degli elementi
    lista_risultati = []

    # Itera finché tutti gli elementi non sono stati selezionati o ratings_copia è vuoto
    while not ratings_copia.empty:
        # Conta il numero di voti per ciascun elemento
        voti_per_elemento = ratings_copia.apply(lambda x: (x == x.max()).astype(int))

        # Somma i voti per ottenere il numero totale di voti per ciascun elemento
        voti_totali = voti_per_elemento.sum(axis=1)

        # Seleziona l'elemento con più voti
        elemento_piu_votato = voti_totali.idxmax()

        # Aggiungi l'elemento più votato alla lista dei risultati
        lista_risultati.append(elemento_piu_votato)

        # Rimuovi l'elemento selezionato dal DataFrame utilizzando l'indice
        ratings_copia = ratings_copia.drop(index=elemento_piu_votato)

    return lista_risultati[:10]

# Applica la strategia di Plurality Voting
risultato = strategia_plurality_voting(normalized_table.T)
print(risultato)


In [ ]:
result_id_normalized = risultato
result_id_normalized

In [ ]:
def get_titles_from_ids(result_id, movies):
    titles = [movies.loc[movies['movieId'] == id_, 'title'].values[0] for id_ in result_id]
    return titles


# Esempio di utilizzo
titles = get_titles_from_ids(result_id_normalized, movies)
print(titles)

In [ ]:
# Confronto se le due liste sono uguali
if result_id == result_id_normalized:
    print("Le liste sono uguali.")
else:
    print("Le liste non sono uguali.")